In [ ]:
import sys
import shutil
sys.path.append('../dependencies/')
import pyemu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import pathlib as pl
import os, platform

## set some global vars we need

In [ ]:
wkdir = pl.Path('.')
template_dir = pl.Path('./tmpdir')
sen_template_dir = pl.Path('./tmpdir_sen')
if template_dir.exists():
    shutil.rmtree(template_dir)
template_dir.mkdir()
if sen_template_dir.exists():
    shutil.rmtree(sen_template_dir)

    
input_yml = 'TestExample.yml'
with open(wkdir / input_yml, 'r') as ifp:
    indat = yaml.safe_load(ifp)
run_global_sen = True
run_ies = False

# Set up `tpl` file for parameterization
### parameterize T and S globally

In [ ]:
T_init = indat['project_properties']['T']
S_init = indat['project_properties']['S']

indat['project_properties']['T'] = f'~{"global_T":^16s}~'
indat['project_properties']['S'] = f'~{"global_S":^16s}~'


### get the starting apportionment values and parameterize them as well

In [ ]:
well_keys = [i for i in indat.keys() if i.startswith('well_')]
app_keys = [[j for j in indat[i].keys() if j.startswith('stream_apportionment')]
                     for i in well_keys]
pending_wells = [i for i in well_keys if 'pending' in indat[i]['status']]
allkeys = dict(zip(well_keys, app_keys))
allkeys

In [ ]:
pending_wells

In [ ]:
pars = list()
parvals = list()
for k,v in allkeys.items():
    for cv in v:
        cpar = f'{k}__{cv}'
        pars.append(cpar)
        parvals.append(indat[k][cv]['apportionment'])
        indat[k][cv]['apportionment'] = f'~{cpar:^45}~'

In [ ]:
pars

In [ ]:
pars_df = pd.DataFrame(index = pars, data = {'parval1':parvals})

In [ ]:
pars_df

In [ ]:
pd.DataFrame(index = ['global_s','global_t'], data = {'parval1':[S_init,T_init]})

In [ ]:
pars_df = pd.concat([pars_df, pd.DataFrame(index = ['global_s','global_t'], data = {'parval1':[S_init,T_init]})])

In [ ]:
pars_df

In [ ]:
with open(template_dir / f"{input_yml}.tpl",'w') as ofp:
    ofp.write('ptf ~\n')
    documents = yaml.dump(indat, ofp, default_flow_style = False, sort_keys= False)

# make `ins` file and external forward run file

In [ ]:
basedeplobs = [f"{indat[k]['name']}:bdpl" for k in indat.keys() if 'stream' in k]

In [ ]:
[i for i in basedeplobs if '93832' in i]

In [ ]:
unique_rivers = list(set([i.split(':')[0] for i in basedeplobs]))

In [ ]:
unique_rivers

In [ ]:
# pending well already included
# basedeplobs.extend([f"{i}:{j.replace('well_','')}:bdpl" for i in unique_rivers for j in pending_wells])

In [ ]:
basedeplobs.extend([f'{i}:{j}:bdpl' for i in unique_rivers for j in ['total_proposed','total_existing','total_combined']])

In [ ]:
with open(template_dir / 'basedeplobs.dat', 'w') as ofp:
    [ofp.write(i + '\n') for i in basedeplobs]

# make forward run script external file

In [ ]:
output_ts = ['TomorrowRiver:92696','TomorrowRiver:70974']
times = range(365*4,365*5+1)

In [ ]:
ts_obs = []
for c_ts in output_ts:
    ts_obs.extend([f'{c_ts}__{i}' for i in times])

In [ ]:
ts_obs

In [ ]:
allobs = basedeplobs + ts_obs

In [ ]:
allobs

In [ ]:
with open(template_dir / 'ts_obs.dat' , 'w') as ofp:
    [ofp.write(c_ts + '\n') for c_ts in output_ts]

In [ ]:
times

In [ ]:
indat

In [ ]:
odir = pl.Path('output')

In [ ]:
base_data = pd.read_csv(odir/f'{input_yml.replace(".yml","")}.table_report.base_stream_depletion.csv', index_col=0)

In [ ]:
base_data

In [ ]:
bdplobs = pd.read_csv(template_dir/'basedeplobs.dat', header=None)
bdplobs.columns = ['obsname']
bdplobs.index = bdplobs.obsname

In [ ]:
bdplobs['obs_values'] = np.nan
bdplobs

In [ ]:
for cob in bdplobs.obsname:
    riv,wel,_ = cob.split(':')
    print(cob)
    bdplobs.loc[cob, 'obs_values'] = base_data.loc[wel][riv]

In [ ]:
bdplobs

In [ ]:
ts_data = pd.read_csv(odir/f'{input_yml.replace(".yml","")}.table_report.all_ts.csv', index_col=0) 

In [ ]:
ts_data.columns = ts_data.columns.str.split('__').str[-1]
#ts_data.columns.str.split('__').str[-1]
ts_data

In [ ]:
ts_path = template_dir / 'ts_obs.dat'
output_ts = [i.strip() for i in open(ts_path, 'r').readlines()]


In [ ]:
ts_df = pd.DataFrame(index = ts_obs, data = {'obsname':ts_obs,'obs_values':np.nan})

In [ ]:
for cob in ts_df.index:
    criv,ctime = cob.split('__')
    ts_df.loc[cob,'obs_values'] = ts_data.loc[int(ctime)][criv]

In [ ]:
ts_df

In [ ]:
allout = pd.concat([bdplobs,ts_df])

In [ ]:
allout['obs_values'].to_csv(template_dir / 'allobs.out', sep = ' ', header=None)

In [ ]:
with open(template_dir / 'allobs.out.ins', 'w') as ofp:
    ofp.write('pif ~\n')
    [ofp.write(f'l1 w !{i}!\n') for i in allout.index]

# Make PST file

In [ ]:
cwd = os.getcwd()

In [ ]:
os.chdir(template_dir)
pst = pyemu.Pst.from_io_files(*pyemu.utils.parse_dir_for_io_files('.'))
os.chdir(cwd)

In [ ]:
pars = pst.parameter_data

In [ ]:
pars.loc[pars_df.index,'parval1'] = pars_df.parval1

In [ ]:
pars

In [ ]:
pars.loc[pars.index.str.startswith('well_'),'parlbnd'] = \
        pars.loc[pars.index.str.startswith('well_'),'parval1']-.1
pars.loc[pars.index.str.startswith('well_'),'parubnd'] = \
        pars.loc[pars.index.str.startswith('well_'),'parval1']+.1


In [ ]:
pars.loc[pars.parlbnd <=0, 'parlbnd'] = 0.01
pars.loc[pars.parubnd >=1, 'parubnd'] = 1
pars.partrans = 'none'

In [ ]:
pars.loc['global_s', 'parlbnd'] = 0.05
pars.loc['global_s', 'parubnd'] = 0.2 # can make these a function of starting values later

pars.loc['global_t', 'parlbnd'] = 0.025 * pars.loc['global_t', 'parval1']
pars.loc['global_t', 'parubnd'] = 12 * pars.loc['global_t', 'parval1']
pars.loc['global_t', 'partrans'] = 'log' 
pars

In [ ]:
pst.control_data.noptmax = -1
pst.model_command = [f'python run_pycap.py {input_yml} ts_obs.dat']
pst.pestpp_options['par_sigma_range'] = 6
pst.pestpp_options['ies_num_reals'] = 500


In [ ]:
pst.write(str(template_dir / 'prior_mc.pst'), version=2)

In [ ]:
shutil.copy2('run_pycap.py', template_dir / 'run_pycap.py')
# shutil.copy2('../dependencies/bin/pestpp-ies', template_dir / 'pestpp-ies')
shutil.copytree('../hicap_analysis/', template_dir / 'hicap_analysis')
shutil.rmtree(template_dir / 'hicap_analysis' / 'tests')

### now make the sensitivity directory

In [ ]:
shutil.copytree(template_dir, sen_template_dir)

## And update the PST file for global sensitivity

In [ ]:
pst_sen = pyemu.Pst(str(sen_template_dir / 'prior_mc.pst'))

In [ ]:
pst_sen.write(str(sen_template_dir / 'global_sens.pst'))

In [ ]:
if run_global_sen:
    if 'window' in platform.platform().lower():
        pestpp_ex = '../../dependencies/win_bin/pestpp-sen'
    else:
        pestpp_ex = '../../dependencies/mac_bin/pestpp-sen'
    pyemu.os_utils.start_workers(
            worker_dir=str(sen_template_dir), exe_rel_path=pestpp_ex,
            pst_rel_path='global_sens.pst', num_workers=20,
            worker_root='./', master_dir='MASTER_SEN')

In [ ]:
if run_ies:
    if 'window' in platform.platform().lower():
        pestpp_ex = '../../dependencies/win_bin/pestpp-ies'
    else:
        pestpp_ex = '../../dependencies/mac_bin/pestpp-ies'
    pyemu.os_utils.start_workers(
            worker_dir=str(template_dir), exe_rel_path=pestpp_ex,
            pst_rel_path='prior_mc.pst', num_workers=None,
            worker_root='./', master_dir='MASTER')